In [7]:
#some random semantics test data, something that I did for first attempt
num_data = 100
dataX = [autograd.Variable(torch.randn((1, 75))) for _ in range(num_data)]
dataY = [autograd.Variable(torch.LongTensor([random.randint(0,9)])) for _ in range(num_data)]
# print(dataY)
# print(dataX)

inp_dim=75
hidden_dim=75
lstm = nn.LSTM(inp_dim, hidden_dim) # Input dim is 3, output dim is 3
inputs = [autograd.Variable(torch.randn((1, inp_dim)))
          for _ in range(10)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (autograd.Variable(torch.randn(1, 1, hidden_dim)),
          autograd.Variable(torch.randn((1, 1, hidden_dim))))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (autograd.Variable(torch.randn(1, 1, hidden_dim)),
          autograd.Variable(torch.randn((1, 1, hidden_dim))))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
#print(hidden)

Variable containing:
(0 ,.,.) = 

Columns 0 to 5 
  -2.6916e-01  7.4629e-03  1.1097e-01 -2.3371e-01  1.2026e-01 -1.1477e-01

Columns 6 to 11 
  -1.0192e-01  1.6501e-01  5.5011e-02  3.6876e-01 -6.9174e-02 -1.1584e-01

Columns 12 to 17 
   3.1950e-03  2.4456e-02 -3.3257e-02  1.5912e-01 -4.0697e-02 -2.7386e-02

Columns 18 to 23 
   4.1282e-01  2.2779e-01 -3.6151e-01 -3.2388e-01  3.2156e-02  1.9831e-01

Columns 24 to 29 
   3.7859e-02  7.7129e-03  4.2650e-02 -5.3880e-02  2.2618e-01  4.1684e-02

Columns 30 to 35 
   3.1258e-01 -3.5921e-01  7.3594e-02 -4.6318e-01  4.9253e-01  2.0950e-02

Columns 36 to 41 
  -1.7698e-01 -3.5666e-02 -1.8573e-02  3.1260e-01 -1.3584e-03 -3.2762e-02

Columns 42 to 47 
   2.8882e-01 -3.1665e-02 -5.8151e-02  6.4970e-02  8.7607e-03 -1.2327e-01

Columns 48 to 53 
   4.8734e-01  3.4143e-01  1.7959e-01  5.2863e-01  2.1109e-02  2.4651e-01

Columns 54 to 59 
   3.0828e-01  1.2297e-01  6.1541e-01  1.5370e-01  5.4931e-01 -9.9900e-02

Columns 60 to 65 
   4.0472e-01  4.6134

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random, numpy as np
import pandas as pd
import matplotlib.pyplot as plt

torch.manual_seed(1)

In [2]:
#loading and prepping data
#initially only one action
dframe = pd.read_csv('./csv_data/action_1.csv')
dframe2 = pd.read_csv('./csv_data/action_2.csv')

In [128]:
dframe.loc[dframe['video_id'].isin([0,1,2])]

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,label,id,video_id
0,-0.382997,-0.419442,3.449989,-0.366909,-0.092619,3.443680,-0.353380,0.229542,3.427116,-0.391862,...,3.636719,-0.435790,-0.536338,3.280097,-0.364369,-0.491436,3.269750,1,72057594037944340,0
1,-0.383146,-0.419292,3.450006,-0.367569,-0.092003,3.443895,-0.353885,0.230300,3.427162,-0.391820,...,3.633053,-0.436031,-0.536649,3.281972,-0.358806,-0.471054,3.269975,1,72057594037944340,0
2,-0.385776,-0.421191,3.449611,-0.369506,-0.092775,3.443796,-0.354571,0.230189,3.426965,-0.403822,...,3.632370,-0.436489,-0.536484,3.286322,-0.358079,-0.470344,3.270202,1,72057594037944340,0
3,-0.385807,-0.421205,3.449582,-0.369576,-0.092714,3.443878,-0.354524,0.230369,3.427140,-0.403580,...,3.499778,-0.441701,-0.533234,3.278971,-0.360298,-0.476572,3.268953,1,72057594037944340,0
4,-0.357840,-0.420304,3.438846,-0.364956,-0.092426,3.442334,-0.354907,0.230391,3.427352,-0.405945,...,3.400878,-0.430001,-0.536492,3.278641,-0.358697,-0.471415,3.270685,1,72057594037944340,0
5,-0.317779,-0.424233,3.417407,-0.347579,-0.091598,3.435570,-0.354990,0.231382,3.428351,-0.413833,...,3.376484,-0.440098,-0.535446,3.284364,-0.360256,-0.475610,3.269022,1,72057594037944340,0
6,-0.340148,-0.420218,3.429176,-0.349129,-0.091376,3.437132,-0.355946,0.231430,3.430860,-0.419386,...,3.402198,-0.431560,-0.535082,3.279142,-0.358869,-0.482122,3.268943,1,72057594037944340,0
7,-0.326968,-0.420078,3.419758,-0.347450,-0.091232,3.436175,-0.357365,0.231702,3.432635,-0.423889,...,3.375357,-0.435553,-0.536029,3.278222,-0.357651,-0.469825,3.270531,1,72057594037944340,0
8,-0.327881,-0.426224,3.441594,-0.346586,-0.092981,3.442161,-0.357856,0.231570,3.434157,-0.423823,...,3.362528,-0.438622,-0.534724,3.280972,-0.359408,-0.476015,3.269827,1,72057594037944340,0
9,-0.375689,-0.436647,3.439764,-0.356871,-0.095212,3.445247,-0.358203,0.232436,3.436680,-0.425678,...,3.341025,-0.432623,-0.537754,3.283831,-0.357795,-0.465352,3.270250,1,72057594037944340,0


In [3]:
#making test and train split
def train_test_split(dframe_list):
    train_split = np.empty(0, dtype=object)
    test_split = np.empty(0, dtype=object)
    for dframe in dframe_list:
        label = dframe.iloc[0,75]
        num_samples = len(dframe.iloc[:,:])
        video_ids = np.unique(dframe.iloc[:,-1].values)
        train_video_ids = video_ids[:-15]
        test_video_ids = video_ids[-15:]
        train_split1 = np.empty(len(train_video_ids), dtype=object)
        test_split1 = np.empty(len(test_video_ids), dtype=object)

        for idx,i in enumerate(train_video_ids):
            train_split1[idx] = dframe.loc[dframe['video_id'] == i].values[:,0:75]
            mean_vec = np.mean(train_split1[idx], axis=0)
            std_vec = np.std(train_split1[idx], axis=0)
            train_split1[idx] = ((train_split1[idx] - mean_vec)/std_vec, label)

        for idx,i in enumerate(test_video_ids):
            test_split1[idx] = dframe.loc[dframe['video_id'] == i].values[:,0:75]
            mean_vec = np.mean(test_split1[idx], axis=0)
            std_vec = np.std(test_split1[idx], axis=0)
            test_split1[idx] = ((test_split1[idx] - mean_vec)/std_vec, label)
        train_split = np.concatenate((train_split, train_split1))
        test_split = np.concatenate((test_split, test_split1))
    
    return train_split, test_split
    #print(train_split[0])
train_split, test_split = train_test_split([dframe, dframe2])

In [ ]:
# train_split

In [4]:
def Data_gen( train_split, SEQ_LEN):
    while(True):
        X = train_split
        databatch = random.sample(list(X), 1)[0]
        databatch, label = databatch[0], databatch[1]
        if len(databatch) > SEQ_LEN:
            databatch = databatch[0:SEQ_LEN]
        elif len(databatch) < SEQ_LEN:
            databatch = np.concatenate((databatch, np.zeros((SEQ_LEN - len(databatch), 75))))
        else:
            pass
        yield databatch,label

ACTd = Data_gen(train_split, 120)

In [5]:
 next(ACTd)

(array([[-0.20499984, -1.98774498,  2.55898443, ...,  1.62420074,
         -2.01481202,  2.32145499],
        [-0.16392269, -1.98841108,  2.42968468, ...,  1.85123807,
         -1.87788513,  2.23028917],
        [-0.03159299, -1.88323331,  1.97890196, ...,  1.69023177,
         -1.9570052 ,  2.25801596],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]), 1)

In [75]:
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [5]:
#action LSTM
class LSTMClassifier(nn.Module):
    def __init__(self, joints_dim, hidden_dim, label_size, batch_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.lstm = nn.LSTM(joints_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # the first is the hidden h
        # the second is the cell  c
        return (autograd.Variable(torch.zeros(120, self.batch_size, self.hidden_dim)),
                autograd.Variable(torch.zeros(120, self.batch_size, self.hidden_dim)))

    def forward(self, joints3d_vec):
        #x = torch.cat(joints3d_vec).view(len(joints3d_vec), self.batch_size, -1)
        x = joints3d_vec
        #print(x.shape)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
#         print('types bitch', x, self.hidden[0], self.hidden[1])
        #print(lstm_out.shape)
        y  = self.hidden2label(lstm_out[-1])
        #print(y.shape)
        log_probs = F.log_softmax(y, dim=1)
        #print(log_probs.shape)
        return log_probs

In [6]:
model0 = LSTMClassifier(75, 75, 10, 1)
# model0 = model0.cuda().type(torch.FloatTensor)

In [7]:
def evaluate_accuracy(model, test_split):
    pred_labels = np.empty(len(test_split))
    orig_labels = np.array([t[1] for t in test_split])
#     Y = autograd.Variable(torch.LongTensor(orig_labels))
    for i in range(len(test_split)):
        d_in = autograd.Variable(torch.from_numpy(test_split[i][0]).float())
        y_pred = model(d_in)
        pred_labels[i] = y_pred.data.max(1)[1].numpy()[0];
    n_samples = len(pred_labels)
    res=(orig_labels==pred_labels)
    correct_count = (res==True).sum()
    #print(correct_count*100/n_samples)
    return (correct_count*100/n_samples)
# acc = evaluate_accuracy(model0, (testX, testL))
# print(acc)

In [ ]:
#training function
def train(model, num_epoch, num_iter, rec_interval, disp_interval):
    optimizer = optim.Adam(model.parameters(), lr = 1e-4)
    loss_values = []
    rec_step = 0
    print('inside train...')
    for eph in range(num_epoch):
        print('epoch {} starting ...'.format(eph))
        avg_loss = 0
        n_samples = 0
        for i in range(num_iter):
            model.hidden = (model.hidden[0].detach(), model.hidden[1].detach())
            model.zero_grad()
            X,Y = next(ACTd)
            n_samples += len(X)
            X = autograd.Variable(torch.from_numpy(X).float())
#             X = X.cuda()
#             X = X.view(120,1,-1)
#             X = X.view(120,1,-1)
            Y = autograd.Variable(torch.LongTensor(np.array([Y])))
#             Y = Y.cuda()
#             print(Y)
#             print(X)
#             print(X.shape)
            y_hat = model(X)       
            loss = F.cross_entropy(y_hat, Y).cuda()
            avg_loss += loss.data.cpu()[0]
            if i % disp_interval == 0:
                print('epoch: %d iterations: %d loss :%g' % (eph, i, loss.data[0]))
            if rec_step%rec_interval==0:
                loss_values.append(loss.data[0])
            loss.backward()     
            optimizer.step()
            rec_step += 1
            
        avg_loss /= n_samples
        #evaluating model accuracy
        acc = evaluate_accuracy(model, test_split)
        print('epoch: {} <====train track===> avg_loss: {}, accuracy: {}% \n'.format(eph, avg_loss, acc))
    return loss_values
l = train(model0, 45, 100, 2, 20)
plt.plot(l)